In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Winter '22/IR/Project/"

 aspectWordMat		    final_df_withsim500K.sav   nb_sim250k.sav
 aspectWordMat_json	    final_df_withsim50k.sav    nb_sim500K.sav
 DT_sim250k.sav		    final_df_withsim.csv       nb_sim.sav
 DT_sim500K.sav		    final_df_withsim.sav       nb_withoutsim500K.sav
 DT_sim.sav		    Full_HotelRec.zip	       ProcessedText.txt
 FetchedRecords.txt	    IR_Proj.ipynb	       RForest_sim250k.sav
 final_df250k.sav	    KNN.sav		       RForest_sim500K.sav
 final_df500k.sav	    KNN_sim250k.sav	       RForest_sim.sav
 final_df500K.sav	    KNN_sim500K.sav	       rp166f-wang.pdf
 final_df50k.sav	    KNN_sim.sav		       svm_sim250k.sav
 final_df.csv		    List_for_ranking100K.sav   svm_sim.sav
 final_df.sav		    Logistic_sim250k.sav       svm_withoutsim500K.sav
 final_df_withsim100K.sav   Logistic_sim500K.sav       UniqueHotels.pkl
 final_df_withsim250k.sav   Logistic_sim.sav	      'Wang_LARA DFo7R.pdf'


In [ ]:
from itertools import islice

PATH = "/content/drive/My Drive/Winter '22/IR/Project/ProcessedText.txt"

N = 500000
with open(PATH,"r") as myfile:
    record_list = list(islice(myfile, N))
print(len(record_list))

500000


In [ ]:
import json
from collections import defaultdict

rev=""
all_rev=[]
all_rev_txt=[]
all_rev_rating=[]
all_rev_id=[]

for r in range(len(record_list)):
  json_data=json.loads(record_list[r])
  rev+=json_data['text']
  all_rev.append(json_data)
  all_rev_txt.append(json_data['text'])
  all_rev_rating.append(json_data['property_dict'])

#Clearing memory
record_list = None

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk import FreqDist
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize

sentences=[]
review_index=[]

ind=0
for review in all_rev_txt:
  temp1=review.split('.')
  temp2=[]
  #print(temp1)
  for s in temp1:
    sentence=s.lower()
    words=word_tokenize(s)
    en_stopwords = set(stopwords.words('english'))
    processed_words=[]
    for token in words:
          if token not in en_stopwords:
              processed_words.append(token)

    tokenizer= RegexpTokenizer(r"\w+")
    word_list=tokenizer.tokenize(' '.join(processed_words))
    word_list=[w.lower() for w in word_list if len(w)!=1]
    temp2.append(word_list)

  temp3=[]
  for s in temp2:
    if(len(s)!=0):
      temp3.append(' '.join(s))
      review_index.append(ind)

  sentences+=temp3
  ind+=1

#sentences
print(len(sentences))

#Clearing Memory
all_rev=None
all_rev_txt=None
all_rev_rating=None
all_rev_id=None

3747557


In [ ]:
#sentences=nltk.sent_tokenize(rev)
sentence_list=[]
for s in sentences:
  sentence=s.lower()
  words=word_tokenize(s)
  en_stopwords = set(stopwords.words('english'))
  processed_words=[]
  for token in words:
        if token not in en_stopwords:
            processed_words.append(token)

  tokenizer= RegexpTokenizer(r"\w+")
  word_list=tokenizer.tokenize(' '.join(processed_words))
  word_list=[w.lower() for w in word_list if len(w)!=1]
  sentence_list.append(word_list)

sentences=[]
for s in sentence_list:
  if(len(s)!=0):
    sentences.append(' '.join(s))

sentence_wordfreq=[]

for i in sentence_list:
  sentence_wordfreq.append(FreqDist(i))

print(sentence_wordfreq)

#Clearing Memory
sentence_list = None

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from typing import DefaultDict
aspect_keyword=DefaultDict()

aspect_keyword["value"]=["value","price"]
aspect_keyword["room"]=["room","space"]
aspect_keyword["location"]=["location","locate"]
aspect_keyword["cleanliness"]=["clean","dirty"]
aspect_keyword["service"]=["service","manager"]

print(aspect_keyword)

defaultdict(None, {'value': ['value', 'price'], 'room': ['room', 'space'], 'location': ['location', 'locate'], 'cleanliness': ['clean', 'dirty'], 'service': ['service', 'manager']})


functions needed for aspectsegmentation

In [ ]:
all_words=[]

for s in sentences:
  for w in s.split():
    if w not in all_words:
      all_words.append(w)

all_words = list(all_words)

In [ ]:
from collections import defaultdict

def assign_aspect(wordfreq):
  sentence_aspect=[]
  count=DefaultDict(int)
  for word in wordfreq.keys():
    for aspect, keywords in aspect_keyword.items():
      if word in keywords:
        count[aspect]+=1
  if count : 
    maxi = max(count.values())
    for aspect, cnt in count.items():
      if cnt==maxi:
        sentence_aspect.append(aspect)

  return sentence_aspect


In [ ]:
import copy

sentences_dup=copy.deepcopy(sentences)

Aspect Segmentation

In [ ]:

from collections import defaultdict
import heapq
import copy

aspectWordMat = defaultdict(lambda: defaultdict(int))
aspectCount= DefaultDict(int)
wordCount=DefaultDict(int)

i=1000
changed=True
while changed:
  i-=1
  sentence_aspect=[]
  for word_freq in sentence_wordfreq:
    sentence_aspect.append(assign_aspect(word_freq))

  aspect_to_sentence=DefaultDict(list)
  aspect_to_sentence.clear()
  for i in range(len(sentences)):
    if len(sentence_aspect[i])==1:
      aspect_to_sentence[sentence_aspect[i][0]].append(sentences[i])

  #print(aspect_to_sentence)

  for aspect, sentences in aspect_to_sentence.items():
              for i in range(len(sentences)):
                  for word,freq in sentence_wordfreq[i].items():
                      aspectWordMat[aspect][word]+=freq
                      aspectCount[aspect]+=freq
                      wordCount[word]+=freq
                      #print(aspect,"-",word)


  top_p_words = {}

  for aspect in aspect_keyword.keys():
    top_p_words[aspect] = []
  for word in all_words:
      maxChi = 0.0 
      maxAspect = "" 
      for aspect in aspect_keyword.keys():
        C= sum(aspectCount.values())
        C1 = aspectWordMat[aspect][word]
        C2 = wordCount[word]-C1
        C3 = aspectCount[aspect]-C1 
        C4 = C-C1 
        deno = (C1+C3)*(C2+C4)*(C1+C2)*(C3+C4)
        if deno!=0:
          chisq = (C*(C1*C4 - C2*C3)*(C1*C4 - C2*C3))/deno
        else:
          chisq = 0.0

        aspectWordMat[aspect][word] = chisq
        if aspectWordMat[aspect][word] > maxChi:
          maxChi = aspectWordMat[aspect][word]
          maxAspect = aspect
      if maxAspect!="":
        top_p_words[maxAspect].append((maxChi, word))

  changed=False
  top_p_words_copy=copy.deepcopy(top_p_words)
  for aspect in aspect_keyword.keys():
    max_words=[]
    for i in range(5):
      max_word=""
      max_val=0
      max_tup=None
      for tup in top_p_words_copy[aspect]:
        if tup[0]>max_val:
          max_val=tup[0]
          max_word=tup[1]
          max_tup=tup
      max_words.append(max_word)
      try:
        top_p_words_copy[aspect].remove(max_tup)
      except:
        pass
    
    for word in max_words:
      if word not in aspect_keyword[aspect]:
        changed=True
        aspect_keyword[aspect].append(word)


    

#aspect_keyword

In [ ]:
print(aspect_keyword.keys())

dict_keys(['value', 'room', 'location', 'cleanliness', 'service'])


In [ ]:
i = 0
for aspect in aspect_keyword:
  i+=1
  if i==8:
    break
  print(aspect, " : ", aspect_keyword[aspect][0:7])

value  :  ['value', 'price', 'riad', 'kayumanis', 'kop', 'montparnasse', 'williams']
room  :  ['room', 'space', 'hotel', 'disneyland', 'mahana', 'starhill', 'cane']
location  :  ['location', 'locate', 'season', 'view', 'ceylin', 'aparts', 'samui']
cleanliness  :  ['clean', 'dirty', 'hyatt', 'convention', 'kayana', 'gb', 'movie']
service  :  ['service', 'manager', 'villa', 'toberua', 'gowrie', 'asheville', 'romeo']


----


In [ ]:
from textblob import TextBlob

sentences=[]
aspects=[]
reviewId=[]
sentiments=[]

for i in range(len(sentence_aspect)):
  if len(sentence_aspect[i])!=0:
    try:
      sentences.append(sentences_dup[i])
      sentiments.append(TextBlob(sentences_dup[i]).sentiment.polarity)
      try:
        aspects.append(sentence_aspect[i][0])    
      except Exception as e:
        print(e, " Skipping...")
      
      reviewId.append(review_index[i])
    except:
      pass


In [ ]:
print(len(sentences_dup))
print(len(sentences))
print(len(sentence_aspect))

3747547
1510362
3747557


In [ ]:
import pandas as pd

final_df_list=[]
for i in range(len(sentences)):
  temp=[reviewId[i],sentences[i],aspects[i],sentiments[i]]
  final_df_list.append(temp)

final_df=pd.DataFrame(final_df_list,columns=['ReviewId','sentence','aspect','sentiment score'])

In [ ]:
import joblib

joblib.dump(final_df, "drive/My Drive/Winter '22/IR/Project/final_df500K.sav")

["drive/My Drive/Winter '22/IR/Project/final_df500K.sav"]

In [ ]:
final_df = joblib.load("drive/My Drive/Winter '22/IR/Project/final_df500K.sav")

In [ ]:
#final_df.to_csv('final_df.csv')
#!cp final_df.csv "drive/My Drive/Winter '22/IR/Project/"

In [ ]:
final_df.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split( final_df['sentence'], final_df['aspect'], test_size=0.33, random_state=42)

In [ ]:
#Clearing memory
final_df = None

naive bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
import joblib

joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/nb_withoutsim500K.sav")

svm

In [ ]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', svm.SVC(decision_function_shape='ovr')),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/svm_withoutsim500K.sav")

In [ ]:
X_train = None
X_test = None
y_train = None
y_test = None

In [ ]:
print(sentences)

with cosine similarity

In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np

aspects_with_cosinesim=[]

for s in sentences:
  for aspect in aspect_keyword.keys():

    max_sim=0
    maxsim_aspect=""
    
    X_set = set(aspect_keyword[aspect]) 
    Y_set = set(s.split())
    l1 =[]
    l2 =[]
     
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: 
          l1.append(1)
        else:
          l1.append(0) 
        if w in Y_set: 
          l2.append(1)
        else:
           l2.append(0)
    c = 0
      
   
    for i in range(len(rvector)):
        c+= l1[i]*l2[i]
    l1 = np.array(l1)
    l2 = np.array(l2)

    cosine = c / float((np.sum(l1)*np.sum(l2))**0.5)
    
    if(cosine>max_sim):
      maxsim_word=aspect
  
  aspects_with_cosinesim.append(maxsim_word)

In [ ]:
final_df_list_withsim=[]
for i in range(len(sentences)):
  temp=[reviewId[i],sentences[i],aspects_with_cosinesim[i],sentiments[i]]
  final_df_list_withsim.append(temp)

final_df_withsim=pd.DataFrame(final_df_list_withsim,columns=['ReviewId','sentence','aspect','sentiment score'])

In [ ]:
final_df_withsim.head()

,ReviewId,sentence,aspect,sentiment score
0,0,decided family holiday destination saw ranking...,location,0.000000
1,0,plus staff extremely kind willing help time gr...,service,0.211199
2,1,great customer service good restaurant service...,service,0.833333
3,1,giulianuova pretty simple wow town clean beach...,cleanliness,0.179167
4,1,baltic simple functional hotel make special pe...,location,0.178571


In [ ]:
joblib.dump(final_df_withsim, "drive/My Drive/Winter '22/IR/Project/final_df_withsim500K.sav")

["drive/My Drive/Winter '22/IR/Project/final_df_withsim500K.sav"]

In [ ]:
#savedf
final_df_withsim.to_csv('final_df_withsim500K.csv')
!cp final_df_withsim.csv "drive/My Drive/Winter '22/IR/Project/"

cp: cannot stat 'final_df_withsim.csv': No such file or directory


In [ ]:
!ls "drive/My Drive/Winter '22/IR/Project/"

ls: cannot access "drive/My Drive/Winter '22/IR/Project/": No such file or directory


In [ ]:
import joblib

final_df_withsim = joblib.load("drive/My Drive/Winter '22/IR/Project/final_df_withsim500K.sav")

FileNotFoundError: ignored

In [ ]:
final_df_withsim.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split( final_df_withsim['sentence'], final_df_withsim['aspect'], test_size=0.33, random_state=42)

In [ ]:
#Clearing memory
final_df_withsim = None
sentences= None
aspects= None
reviewId= None
sentiments= None
aspects_with_cosinesim= None

naive bayes (with cosine sim)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6253240239155732
(0.8108021197734476, 0.43444976400082985, 0.4773614068370164, None)


In [ ]:
import joblib

joblib.dump(nb, 'nb_sim.sav')

['nb_sim.sav']

In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/nb_sim500K.sav")

["drive/My Drive/Winter '22/IR/Project/nb_sim500K.sav"]

In [ ]:
nb = joblib.load('nb_sim.sav')
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
!ls

SVM ( with cosine sim )

In [ ]:
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', svm.SVC(decision_function_shape='ovr')),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

In [ ]:
joblib.dump(nb, 'svm_sim.sav')

In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/svm_sim500K.sav")

In [ ]:
nb = joblib.load('svm_sim.sav')
y_pred = nb.predict(X_test)
print('accuracy %s' % accuracy_score(y_pred, y_test))

In [ ]:
nb.predict(pd.DataFrame([" want clean room"])[0])

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', DecisionTreeClassifier()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.6557682275992135
(0.6427291481399131, 0.5819277171406937, 0.6072017530512868, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/DT_sim500K.sav")

["drive/My Drive/Winter '22/IR/Project/DT_sim500K.sav"]

Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', LogisticRegression()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


accuracy 0.700563781549697
(0.8651317505579353, 0.5768825417167032, 0.655421842012111, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/Logistic_sim500K.sav")

["drive/My Drive/Winter '22/IR/Project/Logistic_sim500K.sav"]

KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', KNeighborsClassifier(n_neighbors=10)),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.5529553388708318
(0.6041064832056328, 0.40768966790619016, 0.44430691796412936, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/KNN_sim500K.sav")

["drive/My Drive/Winter '22/IR/Project/KNN_sim500K.sav"]

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier(n_estimators=50, max_depth=80)),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))

print(precision_recall_fscore_support(y_test, y_pred, average='macro'))
#print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.5788692267565507
(0.8701038481548389, 0.3678101423249022, 0.4005268475302578, None)


In [ ]:
joblib.dump(nb, "drive/My Drive/Winter '22/IR/Project/RForest_sim500K.sav")

["drive/My Drive/Winter '22/IR/Project/RForest_sim500K.sav"]